In [1]:
# Importing the required Libraries
import statistics
import pandas as pd
import scipy.stats as stats
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
import numpy as np
import seaborn as sns

from termcolor import colored

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)

In [2]:
#Importing the Datasets: Two sets are stored in different sheets of excel
DataGroupA = pd.read_excel("/content/ab_testing_data.xlsx" , sheet_name= "Control Group")
DataGroupB = pd.read_excel("/content/ab_testing_data.xlsx" , sheet_name= "Test Group")

In [4]:
DataGroupA.head()

,Impression,Click,Purchase,Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


In [3]:
DataGroupA.shape

(40, 4)

In [5]:
DataGroupA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


In [6]:
DataGroupA.describe()

,Impression,Click,Purchase,Earning
count,40.000000,40.000000,40.000000,40.000000
mean,101711.449068,5100.657373,550.894059,1908.568300
std,20302.157862,1329.985498,134.108201,302.917783
min,45475.942965,2189.753157,267.028943,1253.989525
25%,85726.690349,4124.304129,470.095533,1685.847205
50%,99790.701078,5001.220602,531.206307,1975.160522
75%,115212.816543,5923.803596,637.957088,2119.802784
max,147539.336329,7959.125069,801.795020,2497.295218


In [7]:
DataGroupB.head()

,Impression,Click,Purchase,Earning
0,120103.503796,3216.547958,702.160346,1939.611243
1,134775.943363,3635.082422,834.054286,2929.405820
2,107806.620788,3057.143560,422.934258,2526.244877
3,116445.275526,4650.473911,429.033535,2281.428574
4,145082.516838,5201.387724,749.860442,2781.697521


In [8]:
print("Row * Column:", DataGroupB.shape)
DataGroupB.info()
print("Summary Statistics of the Test Group:", DataGroupB.describe())

Row * Column: (40, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB
Summary Statistics of the Test Group:           Impression        Click    Purchase      Earning
count      40.000000    40.000000   40.000000    40.000000
mean   120512.411758  3967.549761  582.106097  2514.890733
std     18807.448712   923.095073  161.152513   282.730852
min     79033.834921  1836.629861  311.629515  1939.611243
25%    112691.970770  3376.819024  444.626828  2280.537426
50%    119291.300775  3931.359804  551.355732  2544.666107
75%    132050.578933  4660.497911  699.862360  2761.545405
max    158605.920483  6019.695079  889.910460  3171.489708


In [9]:
#Feature Engineering: Conversion Rate
DataGroupA["Conversion Rate"] = DataGroupA["Purchase"]/DataGroupA["Click"]*100
print(DataGroupA.head())
DataGroupB["Conversion Rate"] = DataGroupB["Purchase"]/DataGroupB["Click"]*100
print(DataGroupB.head())

      Impression        Click    Purchase      Earning  Conversion Rate
0   82529.459271  6090.077317  665.211255  2311.277143        10.922870
1   98050.451926  3382.861786  315.084895  1742.806855         9.314152
2   82696.023549  4167.965750  458.083738  1797.827447        10.990583
3  109914.400398  4910.882240  487.090773  1696.229178         9.918600
4  108457.762630  5987.655811  441.034050  1543.720179         7.365721
      Impression        Click    Purchase      Earning  Conversion Rate
0  120103.503796  3216.547958  702.160346  1939.611243        21.829625
1  134775.943363  3635.082422  834.054286  2929.405820        22.944577
2  107806.620788  3057.143560  422.934258  2526.244877        13.834295
3  116445.275526  4650.473911  429.033535  2281.428574         9.225587
4  145082.516838  5201.387724  749.860442  2781.697521        14.416546


In [10]:
# Earning per Purchase
DataGroupA["Earning per Purchase"] = DataGroupA["Earning"]/DataGroupA["Purchase"]
print(DataGroupA.head())
DataGroupB["Earning per Purchase"] = DataGroupB["Earning"]/DataGroupB["Purchase"]
print(DataGroupB.head())

      Impression        Click    Purchase      Earning  Conversion Rate  \
0   82529.459271  6090.077317  665.211255  2311.277143        10.922870   
1   98050.451926  3382.861786  315.084895  1742.806855         9.314152   
2   82696.023549  4167.965750  458.083738  1797.827447        10.990583   
3  109914.400398  4910.882240  487.090773  1696.229178         9.918600   
4  108457.762630  5987.655811  441.034050  1543.720179         7.365721   

   Earning per Purchase  
0              3.474501  
1              5.531229  
2              3.924670  
3              3.482368  
4              3.500229  
      Impression        Click    Purchase      Earning  Conversion Rate  \
0  120103.503796  3216.547958  702.160346  1939.611243        21.829625   
1  134775.943363  3635.082422  834.054286  2929.405820        22.944577   
2  107806.620788  3057.143560  422.934258  2526.244877        13.834295   
3  116445.275526  4650.473911  429.033535  2281.428574         9.225587   
4  145082.516838  

In [11]:
#Data Manipulation: Indication of Groups in A New Variable
DataGroupA["Group"] = "GroupA"
print(DataGroupA.head())
DataGroupB["Group"] = "GroupB"
print(DataGroupB.head())

      Impression        Click    Purchase      Earning  Conversion Rate  \
0   82529.459271  6090.077317  665.211255  2311.277143        10.922870   
1   98050.451926  3382.861786  315.084895  1742.806855         9.314152   
2   82696.023549  4167.965750  458.083738  1797.827447        10.990583   
3  109914.400398  4910.882240  487.090773  1696.229178         9.918600   
4  108457.762630  5987.655811  441.034050  1543.720179         7.365721   

   Earning per Purchase   Group  
0              3.474501  GroupA  
1              5.531229  GroupA  
2              3.924670  GroupA  
3              3.482368  GroupA  
4              3.500229  GroupA  
      Impression        Click    Purchase      Earning  Conversion Rate  \
0  120103.503796  3216.547958  702.160346  1939.611243        21.829625   
1  134775.943363  3635.082422  834.054286  2929.405820        22.944577   
2  107806.620788  3057.143560  422.934258  2526.244877        13.834295   
3  116445.275526  4650.473911  429.033535  22

In [12]:
#Combining the Datasets
Data = pd.concat([DataGroupA, DataGroupB], ignore_index=True)
print(Data.head())
print(Data.tail())

      Impression        Click    Purchase      Earning  Conversion Rate  \
0   82529.459271  6090.077317  665.211255  2311.277143        10.922870   
1   98050.451926  3382.861786  315.084895  1742.806855         9.314152   
2   82696.023549  4167.965750  458.083738  1797.827447        10.990583   
3  109914.400398  4910.882240  487.090773  1696.229178         9.918600   
4  108457.762630  5987.655811  441.034050  1543.720179         7.365721   

   Earning per Purchase   Group  
0              3.474501  GroupA  
1              5.531229  GroupA  
2              3.924670  GroupA  
3              3.482368  GroupA  
4              3.500229  GroupA  
       Impression        Click    Purchase      Earning  Conversion Rate  \
75   79234.911929  6002.213585  382.047116  2277.863984         6.365104   
76  130702.239410  3626.320072  449.824592  2530.841327        12.404437   
77  116481.873365  4702.782468  472.453725  2597.917632        10.046259   
78   79033.834921  4495.428177  425.35910

In [13]:
print(Data)

       Impression        Click    Purchase      Earning  Conversion Rate  \
0    82529.459271  6090.077317  665.211255  2311.277143        10.922870   
1    98050.451926  3382.861786  315.084895  1742.806855         9.314152   
2    82696.023549  4167.965750  458.083738  1797.827447        10.990583   
3   109914.400398  4910.882240  487.090773  1696.229178         9.918600   
4   108457.762630  5987.655811  441.034050  1543.720179         7.365721   
..            ...          ...         ...          ...              ...   
75   79234.911929  6002.213585  382.047116  2277.863984         6.365104   
76  130702.239410  3626.320072  449.824592  2530.841327        12.404437   
77  116481.873365  4702.782468  472.453725  2597.917632        10.046259   
78   79033.834921  4495.428177  425.359102  2595.857880         9.462038   
79  102257.454089  4800.068321  521.310729  2967.518390        10.860486   

    Earning per Purchase   Group  
0               3.474501  GroupA  
1               5

In [14]:
print("Step 1: Testing the Normality Assumption for Conversion Rate: ", "\n")

GroupA_shaphiro_w, GroupA_shaphiro_p = shapiro(DataGroupA["Conversion Rate"])
GroupB_shaphiro_w, GroupB_shaphiro_p = shapiro(DataGroupB["Conversion Rate"])
print("GroupA_shaphiro_p = %.3f, GroupB_shaphiro_p = %.3f" % (GroupA_shaphiro_p, GroupB_shaphiro_p),"\n")

Step 1: Testing the Normality Assumption for Conversion Rate:  

GroupA_shaphiro_p = 0.000, GroupB_shaphiro_p = 0.000 



Mann-Whitney U test

In [15]:
u_value, mannw_test_p = mannwhitneyu(DataGroupA["Conversion Rate"], DataGroupB["Conversion Rate"])
print('u_value=%.3f, mannw_test_p=%.3f' % (u_value, mannw_test_p),"\n")

u_value=459.000, mannw_test_p=0.001 



The Mann-Whitney U test was conducted to compare the conversion rates between Group A and Group B. The test returned a U value of 459.000 and a p-value of 0.001. Since the p-value is less than the significance level of 0.05, we reject the null hypothesis. This indicates a statistically significant difference in conversion rates between the two groups, suggesting that the distribution of conversion rates is different between Group A and Group B

examine the medians

In [16]:
# Calculate the medians of the conversion rates for both groups
median_group_a = DataGroupA["Conversion Rate"].median()
median_group_b = DataGroupB["Conversion Rate"].median()

print("Median Conversion Rate for Group A:", median_group_a)
print("Median Conversion Rate for Group B:", median_group_b)


Median Conversion Rate for Group A: 10.956726754522823
Median Conversion Rate for Group B: 14.618162469826714


Group B has a higher median conversion rate compared to Group A. This suggests that Group B generally performs better in terms of conversion rate.